In [23]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re




In [ ]:
#scrape root directory from physionet MIMICIII waveform (only)
ROOTURL = "https://physionet.org/files/mimic3wdb/1.0"
RecordsRequest = requests.get(ROOTURL + "/RECORDS-waveforms")

In [29]:
#convert request item in Soup obj and into vector of string names
soup = BeautifulSoup(RecordsRequest.text, 'html.parser')
all_records = str(soup).splitlines();

idx = 0;
for record_path in all_records:
    record_name = record_path[3:10]
    
    LayoutRequest = requests.get(ROOTURL + "/" + record_path + record_name+"_layout.hea")
    soup_layout = BeautifulSoup(LayoutRequest.text, 'html.parser')
    layout_str = str(soup_layout)
    signal_matches = ["ABP", "PLETH"] 
    if all( x in layout_str for x in signal_matches):
        print("SUCCESS - "+ record_name)
    else:
        print("FAIL - "+record_name)
        
    #breaking early from for loop while testing scraper
    idx += 1
    if idx == 3000: break

FAIL - 3000003
FAIL - 3000051
FAIL - 3000060
SUCCESS - 3000063
FAIL - 3000065
FAIL - 3000086
FAIL - 3000100
FAIL - 3000103
FAIL - 3000105
FAIL - 3000125
FAIL - 3000126
FAIL - 3000142
FAIL - 3000154
FAIL - 3000189
FAIL - 3000190
FAIL - 3000203
FAIL - 3000221
FAIL - 3000282
FAIL - 3000336
FAIL - 3000358
FAIL - 3000386
SUCCESS - 3000393
SUCCESS - 3000397
SUCCESS - 3000428
SUCCESS - 3000435
FAIL - 3000458
SUCCESS - 3000480
SUCCESS - 3000484
FAIL - 3000497
FAIL - 3000531
FAIL - 3000544
FAIL - 3000577
FAIL - 3000596
FAIL - 3000598
FAIL - 3000611
FAIL - 3000686
SUCCESS - 3000701
FAIL - 3000710
FAIL - 3000713
SUCCESS - 3000714
FAIL - 3000715
FAIL - 3000716
SUCCESS - 3000717
FAIL - 3000724
FAIL - 3000762
FAIL - 3000775
SUCCESS - 3000781
SUCCESS - 3000801
FAIL - 3000834
FAIL - 3000847
FAIL - 3000855
FAIL - 3000858
SUCCESS - 3000860
FAIL - 3000866
FAIL - 3000878
FAIL - 3000879
FAIL - 3000885
FAIL - 3000912
SUCCESS - 3000960
SUCCESS - 3000983
SUCCESS - 3000989
FAIL - 3001002
FAIL - 3001049
FAIL - 

KeyboardInterrupt: 

In [30]:
RecordsRequest

<Response [200]>

In [16]:
all_records

['30/3000003/',
 '30/3000051/',
 '30/3000060/',
 '30/3000063/',
 '30/3000065/',
 '30/3000086/',
 '30/3000100/',
 '30/3000103/',
 '30/3000105/',
 '30/3000125/',
 '30/3000126/',
 '30/3000142/',
 '30/3000154/',
 '30/3000189/',
 '30/3000190/',
 '30/3000203/',
 '30/3000221/',
 '30/3000282/',
 '30/3000336/',
 '30/3000358/',
 '30/3000386/',
 '30/3000393/',
 '30/3000397/',
 '30/3000428/',
 '30/3000435/',
 '30/3000458/',
 '30/3000480/',
 '30/3000484/',
 '30/3000497/',
 '30/3000531/',
 '30/3000544/',
 '30/3000577/',
 '30/3000596/',
 '30/3000598/',
 '30/3000611/',
 '30/3000686/',
 '30/3000701/',
 '30/3000710/',
 '30/3000713/',
 '30/3000714/',
 '30/3000715/',
 '30/3000716/',
 '30/3000717/',
 '30/3000724/',
 '30/3000762/',
 '30/3000775/',
 '30/3000781/',
 '30/3000801/',
 '30/3000834/',
 '30/3000847/',
 '30/3000855/',
 '30/3000858/',
 '30/3000860/',
 '30/3000866/',
 '30/3000878/',
 '30/3000879/',
 '30/3000885/',
 '30/3000912/',
 '30/3000960/',
 '30/3000983/',
 '30/3000989/',
 '30/3001002/',
 '30/300